# Your info

Full name: Arman Heydari

Student ID: 97521252

#Q2

generates a combination of 1 and 0, with length n (input of method).

In [ ]:
# Q2_graded
from random import randint
def generate_random_binary(n):
  result=''
  for i in range(n):
    result+=str(randint(0, 1))
  return result

returns the decimal (integer) value of a binary string number.

In [ ]:
# Q2_graded
def binary_to_decimal(binary_number):
  c = len(binary_number) - 1
  result=0
  for digit in binary_number:
    result += 2**c * int(digit)
    c -= 1
  return result

here we define the fitness function of this problem. it is always a positive value so the best value for fitness is 0, which is a solution of our equation.

In [ ]:
# Q2_graded
def fitness(x, y):
  return abs(2*x*x + y - 59)

b1 and b2 are two binary string numbers with equal length. with this method, we randomly choose a part of these strings and swith them together. so we return 2 children for any input parents.

In [ ]:
# Q2_graded
def two_point_crossover(b1, b2):
  l = len(b1)
  p1 = randint(0, l)
  p2 = randint(0, l)
  if p2<p1:
    p1, p2 = p2,p1
  b3 = b1[0:p1] + b2[p1:p2] + b1[p2:l]
  b4 = b2[0:p1] + b1[p1:p2] + b2[p2:l]
  return b3, b4

with mutation, the algorithm can test various states. b is a binary number and probability is the chance of mutation. 

random.random() returns a float number between 0 and 1. so if the probability is for example 0.1, the random number is less than probability with chance of 0.1. so I used it for occuring mutation.

In [ ]:
# Q2_graded
import random
def mutation(b, probability):
  if random.random() < probability:
    m1 = randint(0, 5)
    if b[m1] == '0':
      b = b[0:m1] + '1' + b[m1+1:]
    else:
      b = b[0:m1] + '0' + b[m1+1:]
  return b

with calling this method for any x,y you can add them to possible answers (population list). and it fitness will calculated and added to fitnesses list, excatly on the same index as population.

In [ ]:
# Q2_graded
def add_chromosome(x, y, population, fitnesses):
  f = fitness(binary_to_decimal(x), binary_to_decimal(y))
  if f not in fitnesses:
    population.append((x, y))
    fitnesses.append(f)
  return population, fitnesses

using this function, we initialize the population randomly with 'binary_length' genes for each chromosome and n random chromosomes on the population.

In [ ]:
# Q2_graded
def random_initialization(n, binary_length):
  population = []
  fitnesses = []
  for i in range(n):
    population, fitnesses = add_chromosome(generate_random_binary(binary_length), generate_random_binary(binary_length), population, fitnesses)
  return population, fitnesses

this is the main function actually. where you can call it with apporpriate hyperparameters and see the result of genetic programming for this problem.

In [ ]:
# Q2_graded
def genetic_programming(number_of_initialize_population, binary_length, max_population, mutation_probability):
  counter = 0
  population, fitnesses = random_initialization(number_of_initialize_population, binary_length)
  while True:
    l = len(population)
    if l>max_population:
      population = population[l-max_population:]
      fitnesses = fitnesses[l-max_population:]
    
    temp = sorted(set(fitnesses))
    best_couple_index = fitnesses.index(temp[0])
    second_best_couple_index = fitnesses.index(temp[1])
    
    x_best, y_best = population[best_couple_index]
    x_second_best, y_second_best = population[second_best_couple_index]

    if temp[0] == 0:
      print("result found after", counter, "generations:")
      print('x , y =', x_best, ",", y_best)
      print("which means x=", binary_to_decimal(x_best),", y=", binary_to_decimal(y_best))
      break
    
    x_new_1, x_new_2 = two_point_crossover(x_best, x_second_best)
    y_new_1, y_new_2 = two_point_crossover(y_best, y_second_best)

    x_new_1 = mutation(x_new_1, mutation_probability)
    x_new_2 = mutation(x_new_2, mutation_probability)
    y_new_1 = mutation(y_new_1, mutation_probability)
    y_new_2 = mutation(y_new_2, mutation_probability)
    
    population, fitnesses = add_chromosome(x_new_1, y_new_1, population, fitnesses)
    population, fitnesses = add_chromosome(x_new_2, y_new_1, population, fitnesses)
    population, fitnesses = add_chromosome(x_new_1, y_new_2, population, fitnesses)
    population, fitnesses = add_chromosome(x_new_2, y_new_2, population, fitnesses)

    counter+=1

run the algorithm:

In [ ]:
# Q2_graded
genetic_programming(number_of_initialize_population=10,
                    binary_length=6,
                    max_population=20,
                    mutation_probability=0.1)

result found after 57 generations:
x , y = 000101 , 001001
which means x= 5 , y= 9


# Q3

some methods are same as last question. with a little different, that we only have x in population not (x,y) for each chromosome.

In [ ]:
# Q3_graded
import random

def generate_random_binary(n):
  result=''
  for i in range(n):
    result+=str(random.randint(0, 1))
  return result


def two_point_crossover(b1, b2):
  l = len(b1)
  p1 = random.randint(0, l)
  p2 = random.randint(0, l)
  if p2<p1:
    p1, p2 = p2,p1
  b3 = b1[0:p1] + b2[p1:p2] + b1[p2:l]
  b4 = b2[0:p1] + b1[p1:p2] + b2[p2:l]
  return b3, b4


def mutation(b, probability):
  if random.random() < probability:
    m1 = random.randint(0, len(b)-1)
    if b[m1] == '0':
      b = b[0:m1] + '1' + b[m1+1:]
    else:
      b = b[0:m1] + '0' + b[m1+1:]
  return b

the fitness function surely different from question that we solved before. here the fitness should decrease with increasing the sum of values that the thief piched up. so we calculate the sum and weight of piched stuff, the we return 1/value_sum if he piched something and if the weights are not higher than the bag's capacity.

for the state that the weights sum is higher than capacity, we return 1 because it is a bad case and not fitted well. other cases are less than 1 and it's ok.

In [ ]:
# Q3_graded
def fitness(x, weights, values, capacity=25):
  weight_sum = 0
  value_sum = 0
  for i in range(len(x)):
    if x[i] == '1':
      weight_sum += weights[i]
      value_sum += values[i]
  if weight_sum>25 or value_sum==0:
    return 1
  return 1.0/value_sum

using this function, we initialize the population randomly with 'binary_length' genes for each chromosome and 'number_of_initialize_population' random chromosomes on the population.

we need values and weights to use fitness function and calculate fitnesses for the initialized population.

In [ ]:
# Q3_graded
def random_initialization(values, weights, number_of_initialize_population, binary_length):
  population = []
  fitnesses = []
  for i in range(number_of_initialize_population):
    x = generate_random_binary(binary_length)
    population.append(x)
    fitnesses.append(fitness(x, weights, values))
  return population, fitnesses

this is the main function actually. where you can call it with apporpriate hyperparameters and of course weights, values lists and see the result of genetic programming for this problem.

In [ ]:
# Q3_graded
def genetic_knapsack(values, weights, number_of_initialize_population, binary_length, max_population, mutation_probability, generations):
  population, fitnesses = random_initialization(values, weights, number_of_initialize_population, binary_length)
  for i in range(generations):
    l = len(population)
    if l>max_population:
      population = population[l-max_population:]
      fitnesses = fitnesses[l-max_population:]

    temp = sorted(set(fitnesses))
    x_best = population[fitnesses.index(temp[0])]
    x_second_best = population[fitnesses.index(temp[1])]

    x_new_1, x_new_2 = two_point_crossover(x_best, x_second_best)

    x_new_1 = mutation(x_new_1, mutation_probability)
    x_new_2 = mutation(x_new_2, mutation_probability)
    
    if x_new_1 not in population:
      population.append(x_new_1)
      fitnesses.append(fitness(x_new_1, weights, values))
    if x_new_2 not in population:
      population.append(x_new_2)
      fitnesses.append(fitness(x_new_2, weights, values))
  return x_best

run the algorithm with informations given by the question:

In [ ]:
# Q3_graded
VALUES = [30, 10, 20, 50, 70, 15, 40, 25]
WEIGHTS = [2, 4, 1, 3, 5, 1, 7, 4]
NAMES = ["zomorod", "noqre", "yaqut", "almas", "berellian", "firuze", "aqiq", "kahroba"]

best_result = genetic_knapsack(VALUES,
                                WEIGHTS,
                                number_of_initialize_population=5,
                                binary_length=8,
                                max_population=10,
                                mutation_probability=0.1,
                                generations=1000
                               )

print("best chromosome:", best_result, " \n ------------------")
for i in range(len(best_result)):
   if best_result[i]=='1':
     print(NAMES[i])

best chromosome: 10111111  
 ------------------
zomorod
yaqut
almas
berellian
firuze
aqiq
kahroba


# Q4


at the beginning of particle swarm algorithm, we need to initialize particles and their velocities randomly. we do this here with float numbers between 0 and 1. 

pbest and gbest will caculated on next steps, we just initialize them with infinite float numbers.

In [90]:
# Q4_graded
import numpy as np
def random_initialization(particles_no, mlp_weights_no):
  particles = np.random.rand(particles_no, mlp_weights_no)
  velocities = np.random.rand(particles_no, mlp_weights_no)
  pbest = []
  gbest = (particles[0], float('inf'))
  for p in particles:
    pbest.append((p, float('inf')))
  return particles, velocities, pbest, gbest

this method is the loss function for mlp. which means the fitness function for particle swarm optimization. takes the array of predicted y and the truth y values. the returns mean squared error for them.

In [91]:
# Q4_graded
def mean_squared_error(y_predict, y):
  return np.mean((y_predict - y) ** 2)

we may need relu activation function for hidden layer.

In [92]:
# Q4_graded
def relu(x):
  x[x<0] = 0
  return x

each particle is an array of weights. we need to convert this array to the exact layer so we can do the forward propagation. for example for a particle with 18 numbers, the first 12 numbers are hidden layer with shape (6,2) and the output layer is the last 6 numbers with shape (6,1).

In [93]:
# Q4_graded
def particle_to_layer(particle, units=6):
  input_length = int(particle.shape[0]/units) - 1
  hidden_layer = np.zeros((units, input_length))
  output_layer = np.zeros((units, 1))
  for i in range(units):
    for j in range(input_length):
      hidden_layer[i, j] = particle[i*input_length+j]
  
  for i in range(units):
    output_layer[i] = particle[units*input_length+i]
  
  return hidden_layer, output_layer

we can predict the output of train data for each particle using this method.

In [94]:
# Q4_graded
def predict(particle, x_train):
  hidden_layer, output_layer = particle_to_layer(particle)
  z = np.dot(x_train, hidden_layer.T)
  y_predict = np.dot(z, output_layer)
  return y_predict

at each epoch, we need to update gbest = global lowest fitness and pbest = local lowest fitness. this will happen in this method.

In [95]:
# Q4_graded
def update_bests(particles, fitnesses, pbest, gbest):
  for i in range(len(fitnesses)):
    p, f = pbest[i]
    if f>=fitnesses[i]:
      pbest[i] = (particles[i], fitnesses[i])

    if gbest[1]>=fitnesses[i]:
      gbest = (particles[i], fitnesses[i])
  return pbest, gbest

the must important steps on particle swarm optimization is updating the velocities vector. we call this method at the end of each epoch and find the new velocities.

In [97]:
# Q4_graded
from random import random
def update_velocities(particles, velocities, w, c1, c2, pbest, gbest):
  new_velocities = np.zeros(velocities.shape)
  for i in range(velocities.shape[0]):
    delta_p = pbest[i][0] - particles[i]
    delta_g = gbest[0] - particles[i]
    new_velocities[i] = w*velocities[i] + c1*random()*delta_p + c2*random()*delta_g
  return new_velocities

each particle must update with respect to the calculated velocity vector. so we pass the particles array and their velocities to this method, and it returns the new particles.

In [98]:
# Q4_graded
def update_particles(particles, new_velocities):
  new_particles = particles + new_velocities
  return new_particles

this method calls each of the methods in the correct way. so with this function we can train our mlp neural network using particle swarm algorithm with passing the apporpriate hyperparameters beside the train data.

In [150]:
# Q4_graded
def train_mlp(x_train, y_train, epochs, w, c1, c2, particles_no, mlp_weights_no):
  particles, velocities, pbest, gbest = random_initialization(particles_no, mlp_weights_no)
  for epoch in range(epochs):
    fitnesses = []
    for particle in particles:
      y_predict = predict(particle, x_train)
      fitnesses.append(mean_squared_error(y_predict, y_train))
    pbest, gbest = update_bests(particles, fitnesses, pbest, gbest)
    print('best fitness on epoch', epoch+1, 'is:', int(gbest[1]))
    velocities = update_velocities(particles, velocities, w, c1, c2, pbest, gbest)
    particles = update_particles(particles, velocities)
  return gbest[0]

 now we can run algorithm for each given function and evaluate the loss and accuracy for them:
*   y=x



In [122]:
# Q4_graded
x_train = np.random.randint(-10000, 10000, (10000, 1))
y_train = x_train
x_test = np.random.randint(-10000, 10000, (100, 1))
y_test = x_test
best_particle = train_mlp(x_train,
                          y_train,
                          epochs = 20,
                          w = 0.5,
                          c1 = 3,
                          c2 = 1,
                          particles_no = 200,
                          mlp_weights_no= 12
                )
print('------------------------------------------------------')
y_predict_train = np.rint(predict(best_particle, x_train))
print("accuracy on train:", 100 - 100 * (np.mean(y_predict_train != y_train)), '%')
y_predict_test = np.rint(predict(best_particle, x_test))
print("accuracy on test:", 100 - 100 * (np.mean(y_predict_test != y_test)), '%')

best fitness on epoch 1 is: 543
best fitness on epoch 2 is: 543
best fitness on epoch 3 is: 543
best fitness on epoch 4 is: 543
best fitness on epoch 5 is: 543
best fitness on epoch 6 is: 543
best fitness on epoch 7 is: 59
best fitness on epoch 8 is: 59
best fitness on epoch 9 is: 59
best fitness on epoch 10 is: 59
best fitness on epoch 11 is: 56
best fitness on epoch 12 is: 56
best fitness on epoch 13 is: 5
best fitness on epoch 14 is: 5
best fitness on epoch 15 is: 5
best fitness on epoch 16 is: 0
best fitness on epoch 17 is: 0
best fitness on epoch 18 is: 0
best fitness on epoch 19 is: 0
best fitness on epoch 20 is: 0
------------------------------------------------------
accuracy on train: 100.0 %
accuracy on test: 100.0 %




*   y = x1 + x2


In [130]:
# Q4_graded
x_train = np.random.randint(-10000, 10000, (10000, 2))
y_train = np.sum(x_train, axis=1, keepdims=True)
x_test = np.random.randint(-10000, 10000, (100, 2))
y_test = np.sum(x_test, axis=1, keepdims=True)
best_particle = train_mlp(x_train,
                          y_train,
                          epochs = 50,
                          w = 0.5,
                          c1 = 3,
                          c2 = 1,
                          particles_no = 500,
                          mlp_weights_no= 18
                )
print('------------------------------------------------------')
y_predict_train = np.rint(predict(best_particle, x_train))
print("accuracy on train:", 100 - 100 * (np.mean(y_predict_train != y_train)), '%')
y_predict_test = np.rint(predict(best_particle, x_test))
print("accuracy on test:", 100 - 100 * (np.mean(y_predict_test != y_test)), '%')

best fitness on epoch 1 is: 7497
best fitness on epoch 2 is: 7497
best fitness on epoch 3 is: 7497
best fitness on epoch 4 is: 7497
best fitness on epoch 5 is: 7497
best fitness on epoch 6 is: 7497
best fitness on epoch 7 is: 5929
best fitness on epoch 8 is: 3162
best fitness on epoch 9 is: 3162
best fitness on epoch 10 is: 796
best fitness on epoch 11 is: 796
best fitness on epoch 12 is: 796
best fitness on epoch 13 is: 223
best fitness on epoch 14 is: 223
best fitness on epoch 15 is: 219
best fitness on epoch 16 is: 182
best fitness on epoch 17 is: 182
best fitness on epoch 18 is: 154
best fitness on epoch 19 is: 154
best fitness on epoch 20 is: 51
best fitness on epoch 21 is: 51
best fitness on epoch 22 is: 43
best fitness on epoch 23 is: 6
best fitness on epoch 24 is: 6
best fitness on epoch 25 is: 0
best fitness on epoch 26 is: 0
best fitness on epoch 27 is: 0
best fitness on epoch 28 is: 0
best fitness on epoch 29 is: 0
best fitness on epoch 30 is: 0
best fitness on epoch 31 is: 

* y = sin(x1) + cos(x2)

In [139]:
# Q4_graded
from math import sin, cos

x_train = np.random.randint(-10000, 10000, (10000, 2))
y_train = np.zeros((10000, 1))
for i in range(x_train.shape[0]):
  y_train[i] = sin(x_train[i,0]) + cos(x_train[i,1])

x_test = np.random.randint(-10000, 10000, (100, 2))
y_test = np.zeros((100, 1))
for i in range(x_test.shape[0]):
  y_test[i] = sin(x_test[i,0]) + cos(x_test[i,1])

best_particle = train_mlp(x_train,
                          y_train,
                          epochs = 100,
                          w = 0.5,
                          c1 = 3,
                          c2 = 1,
                          particles_no = 200,
                          mlp_weights_no= 18
                )

best fitness on epoch 1 is: 20260400.71519408
best fitness on epoch 2 is: 20260400.71519408
best fitness on epoch 3 is: 23087.722293237017
best fitness on epoch 4 is: 14224.95114302179
best fitness on epoch 5 is: 12943.574263836608
best fitness on epoch 6 is: 667.302753425106
best fitness on epoch 7 is: 667.302753425106
best fitness on epoch 8 is: 667.302753425106
best fitness on epoch 9 is: 667.302753425106
best fitness on epoch 10 is: 667.302753425106
best fitness on epoch 11 is: 667.302753425106
best fitness on epoch 12 is: 40.34102166078792
best fitness on epoch 13 is: 40.34102166078792
best fitness on epoch 14 is: 40.34102166078792
best fitness on epoch 15 is: 40.34102166078792
best fitness on epoch 16 is: 40.34102166078792
best fitness on epoch 17 is: 40.34102166078792
best fitness on epoch 18 is: 36.48012109354243
best fitness on epoch 19 is: 36.48012109354243
best fitness on epoch 20 is: 34.94813972395849
best fitness on epoch 21 is: 34.94813972395849
best fitness on epoch 22 i

In [141]:
# Q4_graded
print("the numbers are float here so we can't take accuracy as a good method because the network can't predict the exact value!")
y_predict_train = predict(best_particle, x_train)
print("mean square error on train:", mean_squared_error(y_predict_train, y_train))
y_predict_test = np.rint(predict(best_particle, x_test))
print("mean square error on test:", mean_squared_error(y_predict_test, y_test))

the numbers are float here so we can't take accuracy as a good method because the network can't predict the exact value!
mean square error on train: 0.9956675467425754
mean square error on test: 1.027357302671627




*   y = x1 * x2

this mlp network doesn't have enough capacity to fit on this function. so the result isn't satisfying.


In [170]:
# Q4_graded
x_train = np.random.randint(-10, 10, (100, 2))
y_train = np.prod(x_train, axis=1, keepdims=True)

x_test = np.random.randint(-10, 10, (10, 2))
y_test = np.prod(x_test, axis=1, keepdims=True)

best_particle = train_mlp(x_train,
                          y_train,
                          epochs = 100,
                          w = 0.5,
                          c1 = 3,
                          c2 = 1,
                          particles_no = 100,
                          mlp_weights_no= 18
                )
print('------------------------------------------------------')
y_predict_train = np.rint(predict(best_particle, x_train))
print("accuracy on train:", 100 - 100 * (np.mean(y_predict_train != y_train)), '%')
y_predict_test = np.rint(predict(best_particle, x_test))
print("accuracy on test:", 100 - 100 * (np.mean(y_predict_test != y_test)), '%')

best fitness on epoch 1 is: 1372
best fitness on epoch 2 is: 1372
best fitness on epoch 3 is: 1245
best fitness on epoch 4 is: 1241
best fitness on epoch 5 is: 1241
best fitness on epoch 6 is: 1240
best fitness on epoch 7 is: 1240
best fitness on epoch 8 is: 1240
best fitness on epoch 9 is: 1240
best fitness on epoch 10 is: 1240
best fitness on epoch 11 is: 1240
best fitness on epoch 12 is: 1240
best fitness on epoch 13 is: 1240
best fitness on epoch 14 is: 1240
best fitness on epoch 15 is: 1240
best fitness on epoch 16 is: 1240
best fitness on epoch 17 is: 1240
best fitness on epoch 18 is: 1240
best fitness on epoch 19 is: 1240
best fitness on epoch 20 is: 1240
best fitness on epoch 21 is: 1240
best fitness on epoch 22 is: 1240
best fitness on epoch 23 is: 1240
best fitness on epoch 24 is: 1240
best fitness on epoch 25 is: 1240
best fitness on epoch 26 is: 1240
best fitness on epoch 27 is: 1240
best fitness on epoch 28 is: 1240
best fitness on epoch 29 is: 1240
best fitness on epoch 3

# <font color='red'>Submission</font>

1. Sign up in [Gradescope](https://www.gradescope.com) with proper name and student ID and use the following code to join the class: <font color='red'>**4PZD84**</font>
2. Fill in your full name (seperated by single spaces) and student ID in the beginning of this notebook.
3. After you're done with this notebook, you should do the following:
  - Clear all outputs of the notebook.
  ![clear all outputs](https://i.ibb.co/y6FrttB/Screen-Shot-2021-03-21-at-01-51-42.png)
  - Run all of the cells (if you skipped a question just leave the cell unchanged), and make sure all of your outputs are correct.
  ![run all](https://i.ibb.co/cgRcBZ0/Screen-Shot-2021-03-21-at-01-54-58.png)
  - Save your notebook.
  
  - If you're using Colab, download your notebook.
  ![download ipynb](https://i.ibb.co/2KxYM6K/Screen-Shot-2021-03-21-at-02-03-50.png)
  
  - Put the notebook file you just downloaded and `convert.py` in the same folder run the following command:
  ```bash
  python convert.py
  ```
  This will export your code for each question into a `.py` file.
    - **Note**: if you want to add more cells, add this to the **first** line of the cell:
  ```python
  # Q1_graded
  ```
  according to the question number.
  - There are 2 assignments in Gradescope:

    ![assignments](https://i.ibb.co/10GMhGM/Screen-Shot-2021-03-21-at-02-16-25.png)
  
    You should upload your **codes** and your **notebook** in `HW1` section and your final report for all of the questions as a **single pdf** file in `HW5 - Report`. Autograder will automatically check for:
    - `CI001_HW5.ipynb`
    - `Q1.py`
    - `Q2.py`
    - `Q3.py`
    - Your name and ID in the beginning of `.ipynb` file.

    It is important that you <font color='red'>**don't**</font> change the names of these files before submission.

4. If you pass the autograder, you're good to go.